# Laboratorium Automatyka Pojazdowa: system wizyjny rozpoznający znaki drogowe

## Część 3: Detekcja

Problem do rozwiązania w ramach tego laboratorium jest zadaniem detekcji: najpierw lokalizacji, a następnie - dla każdego zlokalizowanego obiektu - jego klasyfikacji.

W tym laboratorium kluczowe będzie stworzenie pipelineu modeli opracowanych na dwóch poprzednich laboratoriach, aby otrzymać algorytm rozwiązujący problem detekcji znaków drogowych. Stworzone wcześniej modele zostaną zaczytane z katalogu `models`.

Miejsca w kodzie pozostawione do uzupełnienia zostały w tym notatniku oznaczone `# TODO`.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from typing import *
import pickle

## Zaczytanie datasetu

In [ ]:
dataset = tf.data.Dataset.load("../data/localization-test-dataset")

## Załadowanie modelu klasyfikatora

In [ ]:
classifierModel = tf.keras.models.load_model("../models/classifier.keras")

In [ ]:

# potrzebujemy także mapy klas na ich etykiety (nazwy)
with open("../models/classifierClassNames.pickle", "rb") as f:
    classNames = pickle.load(f)
display(classNames)

## Załadowanie modelu lokalizatora

In [ ]:
localizerImported = tf.saved_model.load("../models/localizer")
localizerFunction = localizerImported.signatures['serving_default']

In [ ]:
def drawBB(image: np.ndarray, text: str, color: Tuple[int, int, int], BB: Tuple[int, int, int, int], scale: float):
    x, y, w, h = BB

    cv2.rectangle(
        image,
        # TODO: poniżej należy przekopiować implementację drawBB z poprzedniego zadania:
        # - podać jako pt1 współrzędne lewego górnego rogu BB
        # - podać jako pt2 współrzędne prawego dolnego rogu BB
        # pt1 = ...,
        # pt2 = ...,
        pt1=(x, y),
        pt2=(x + w, y + h),
        color=color,
        thickness=int(10 * scale)
    )
    cv2.putText(
        image,
        text=text,
        org=(x, y - int(40 * scale)),
        fontFace=cv2.LINE_AA,
        fontScale=1.3 * scale,
        color=(255, 255, 255),
        thickness=int(6 * scale)
    )

In [ ]:
plt.figure(figsize=(20, 5))

CLASSIFIER_IMG_SIZE = 256 # px

for i, (image, _) in enumerate(dataset.take(3)):
    plt.subplot(1, 3, i + 1)

    image = (image.numpy() * 255).astype(np.uint8)
    result = localizerFunction(inputs=np.array([image]))

    preview = image.copy()

    for di, (box, locClass, locScore) in enumerate(
        zip(
            result['detection_boxes'][0].numpy(),
            result['detection_classes'][0].numpy().astype(int),
            result['detection_scores'][0].numpy()
        )
    ):
        # localization only has one class (Sign = 1) and we need to filter only "sure" objects
        # TODO: ponizej - upewnic sie, ze klasa z wyjscia modelu lokalizatora to 1, a wynik detekcji jest powyzej sensownego progu (uwaga: wartosc jest znormalizowana)
        if locScore >= ... and locClass == ...:
            ymin, xmin, ymax, xmax = [... for c in box] # TODO: nalezy zaokrąglic wszystkie wspolrzedne z bounding boxa i skonwertować je do typu int - opencv akceptuje liczby calkowite
            w = xmax - xmin
            h = ymax - ymin

            cutOutSign = image[...] # TODO: należy "wyciąć" fragment obrazu o współrzędnych x od xmin do xmax oraz y od ymin do ymax
            # our classifier has a fixed input size of CLASSIFIER_IMG_SIZE x CLASSIFIER_IMG_SIZE, so we need to resize the cut-out image to match it
            cutOutSign = cv2.resize(cutOutSign, (CLASSIFIER_IMG_SIZE, CLASSIFIER_IMG_SIZE), interpolation = cv2.INTER_CUBIC)

            classifierPreds = classifierModel.predict(np.array([cutOutSign]))[0]
            classPredictionIndex = ... # TODO: należy wybrać indeks elementu (wyniku) o największej wartości; ponieważ występują w kolejności zgodnej z indeksami klas, będzie to również indeks klasy
            classPredictionProbability = classifierPreds[classPredictionIndex]
            classPredictionLabel = classNames[classPredictionIndex]

            drawBB(
                preview,
                text=f"{classPredictionLabel}: loc {locScore * 100 :.0f}%, clsf: {classPredictionProbability * 100 :.0f}%",
                color=(0, 255, 0),
                BB=[xmin, ymin, w, h],
                scale=0.4
            )

    plt.imshow(preview)

plt.show()

## Obserwacje i wnioski

...